In [153]:
import pandas as pd
import numpy as np
import geopandas as gpd
import tqdm
import datetime
# read csv
IPC_new = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\IPC_new.csv')

# read comprehensive dataset
comprehensive = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\comprehensive.csv')

# read precipitation data
precipitation = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\precipitation_sum.csv')
# convert date to datetime
IPC_new['date'] = pd.to_datetime(IPC_new['date'])
comprehensive['date'] = pd.to_datetime(comprehensive['date'])
precipitation['date'] = pd.to_datetime(precipitation['date'])

# merge comprehensive and precipitation based on date, lat and lon
comprehensive = comprehensive.merge(precipitation, on=['date', 'lat', 'lon'], how='left')


import pandas as pd
import geopandas as gpd
from shapely import wkt

# Assuming df is your DataFrame and it has a 'geometry' column
comprehensive['geometry'] = comprehensive['geometry'].apply(wkt.loads)

# Now we can convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(comprehensive, geometry='geometry')

# Assuming df is your DataFrame and it has a 'geometry' column
IPC_new['geometry'] = IPC_new['geometry'].apply(wkt.loads)

# Now we can convert the DataFrame to a GeoDataFrame
IPC_new = gpd.GeoDataFrame(IPC_new, geometry='geometry')

# split into list accoridng to date
IPC_new_list = [x for _, x in IPC_new.groupby('date')]
gdf_new_list = [x for _, x in gdf.groupby('date')]

#define a new empty list called comprehensive_new_list
comprehensive_new_list = [None] * len(IPC_new_list)

# spatial merge IPC_new and comprehensive based on date
for i in tqdm.tqdm(range(len(IPC_new_list))):
    comprehensive_new_list[i] = gpd.sjoin(gdf_new_list[i],IPC_new_list[i], how='left', op='intersects')
# concatenate the list into a new dataframe
comprehensive_new = pd.concat(comprehensive_new_list)
#rename date_left as date, and drop date_right
comprehensive_new = comprehensive_new.rename(columns={'date_left':'date'})
comprehensive_new = comprehensive_new.drop(columns=['date_right'])
# drop index_right
comprehensive_new = comprehensive_new.drop(columns=['index_right'])

# assign comprehensive_new to new_comprehensive
new_comprehensive = comprehensive_new
# split into list accoridng to date
new_comprehensive_list = [x for _, x in new_comprehensive.groupby('date')]

# define a new empty list called new_comprehensive_new_list
new_comprehensive_new_list = [None] * len(new_comprehensive_list)

# loop over dates, force geometry to be string
for i in tqdm.tqdm(range(len(new_comprehensive_list))):
    new_comprehensive_new_list[i] = new_comprehensive_list[i].astype({'geometry': 'str'})
# concatenate the list into a new dataframe
new_comprehensive_new = pd.concat(new_comprehensive_new_list)
new_comprehensive = new_comprehensive_new
# reroder the observations, first by date then by geometry
new_comprehensive = new_comprehensive.sort_values(by=['date', 'geometry'])

# give each unique geometry an id
new_comprehensive['area_id'] = new_comprehensive.groupby('geometry').ngroup()
# convert area_id to int
new_comprehensive['area_id'] = new_comprehensive['area_id'].astype(int)
# extract year and month
new_comprehensive['year'] = new_comprehensive['date'].dt.year
new_comprehensive['month'] = new_comprehensive['date'].dt.month
# drop country and title
new_comprehensive = new_comprehensive.drop(columns=['country', 'title'])
# read lat_lon data
lat_lon = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\lat_lon.csv')
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import tqdm
import datetime
import time

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

result = pd.DataFrame(columns=[ 'lat', 'lon', 'country','state','country_code'])
for i in tqdm.tqdm(range(len(lat_lon))):
    # request the data from the API
    url = 'https://geocode.maps.co/reverse?lat=' + str(lat_lon['lat'][i]) + '&lon=' + str(lat_lon['lon'][i]) 
    try:
        response = requests_retry_session().get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    data = response.json()
    # create a new df to store the data
    df = pd.DataFrame(columns=[ 'lat', 'lon', 'country','state','country_code'])
    # extract time and temperature_2m_mean from the json file
    df.loc[0,'lat'] = lat_lon['lat'][i]
    df.loc[0,'lon'] = lat_lon['lon'][i]
    #leave blank if no data
    if data['address']['country'] == None:
        continue
    else:
        df.loc[0,'country'] = data['address']['country']
    if 'state' in data['address']:
        if data['address']['state'] == None:
            continue
        else:
            df.loc[0,'state'] = data['address']['state']
    if data['address']['country_code'] == None:
        continue
    else:
        df.loc[0,'country_code'] = data['address']['country_code']
    # concat the data
    result = pd.concat([result, df], axis=0)
    # sleep for 1 second
    time.sleep(0.5)


# merge result with comprehensive_new
new_comprehensive = new_comprehensive.merge(result, on=['lat', 'lon'], how='left')
# reorder the columns
new_comprehensive = new_comprehensive[['area_id', 'country', 'country_code','state', 'lat','lon', 'year','month','date','estimated_population','overall_phase','phase3_worse_population',
       'phase3_worse_percentage', 'phase1_population', 'phase1_percent',
       'phase2_population', 'phase2_percent', 'phase3_population',
       'phase3_percent', 'phase4_population', 'phase4_percent',
       'phase5_population', 'phase5_percent', 'phase3_worse_percentage_manual',
       'phase3_plus_phase4', 'phase2_worse_percentage_manual',
       'phase2_plus_phase3', 'phase2_plus_phase3_plus_phase4','event_count_battles', 'event_count_explosions',
       'event_count_violence', 'fatalities_battles', 'fatalities_explosions',
       'fatalities_violence', 'GOSIF_GPP', 'rainfall_chirps', 'GOSIF_GPP_SD',
       'elevation', 'soil', 'price', 'temperature_2m_mean',
       'temperature_2m_mean_sd', 'shortwave_radiation_sum',
       'shortwave_radiation_sum_sd', 'precipitation_sum',
       'precipitation_sum_sd','geometry']]


  0%|          | 0/66 [00:00<?, ?it/s]c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):

In [182]:


#extract date, lat and lon
location = new_comprehensive[['area_id','date', 'lat', 'lon']]
# import FAO_price data xlsx file
FAO_price = pd.read_excel(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\FAO_price_data\FAO_DATA_FOR_MIGUEL.xlsx')

# keep only date, lat, lon, price and indexReferencePrice
FAO_price = FAO_price[['date', 'lat', 'long', 'price', 'indexReferencePrice']]
# new price index is price/indexReferencePrice*100
FAO_price['price_index'] = FAO_price['price']/FAO_price['indexReferencePrice']*100

# average on each date and location
FAO_price = FAO_price.groupby(['date', 'lat', 'long']).mean().reset_index()
#drop indexReferencePrice and price
FAO_price = FAO_price.drop(columns=['indexReferencePrice', 'price'])
import pandas as pd
from haversine import haversine, Unit
from sklearn.neighbors import BallTree


# unique all dates from location
dates = location['date'].unique()

# create a empty dataframe to store the data:date	lat	lon	nearest_neighbor_distance	market_lat	market_lon	price_index
market_match = pd.DataFrame(columns=['area_id','date', 'lat', 'lon', 'nearest_neighbor_distance', 'market_lat', 'market_lon', 'price_index'])

def to_radians(degrees):
    return degrees * (np.pi / 180)

# loop over all dates
for date in tqdm.tqdm(dates):
    # filter location by selecting the first date
    location_temp = location[location['date'] == date]
    # filter FAO_price by selecting the first date
    FAO_price_temp = FAO_price[FAO_price['date'] == date]
    # Function to convert degrees to radians
    # Convert latitude and longitude to radians
    coords1_rad = location_temp[['lat', 'lon']].apply(to_radians)
    coords2_rad = FAO_price_temp[['lat', 'long']].apply(to_radians)

    # Create a BallTree
    tree = BallTree(coords2_rad, metric='haversine')

    # Query the BallTree to find the index of the nearest neighbor in set2 for each point in set1
    distances, indices = tree.query(coords1_rad, k=1)

    # Add the nearest neighbor information to df1
    location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
    location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km

    #reset index and add a new column index
    FAO_price_temp = FAO_price_temp.reset_index(drop=True)
    FAO_price_temp['index'] = FAO_price_temp.index

    # Merge the nearest neighbors from df2 into df1
    result = pd.merge(location_temp, FAO_price_temp, left_on='nearest_neighbor_index', right_on='index', suffixes=('_df1', '_df2'))
    # rename date_df1 to date, lat_df1 to lat, lat_df2 to market_lat, long to market_lon, drop nearest_neighbor_index,date_df2,index
    result = result.rename(columns={'date_df1': 'date', 'lat_df1': 'lat', 'lat_df2': 'market_lat', 'long': 'market_lon'})
    result = result.drop(columns=['nearest_neighbor_index', 'date_df2', 'index'])
    #concat result with market_match
    market_match = pd.concat([market_match, result], axis=0)


# save comprehensive_merge to csv
comprehensive_merge.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\comprehensive_merge.csv', index=False)
#also save new_comprehensive to csv
new_comprehensive.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\new_comprehensive_temp.csv', index=False)
# save market_match to csv
market_match.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\market_match.csv', index=False)

  0%|          | 0/66 [00:00<?, ?it/s]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_19532\639303970.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_19532\639303970.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km
C:\Users\WeilunShi\AppData\Local\Temp\ipyk